In [1]:
import sys
import numpy as np
import sklearn as sk
import csv
from sklearn.model_selection import train_test_split
import pycrfsuite
import sklearn_crfsuite
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter
from itertools import chain
import time
from tqdm import tqdm
import pandas as pd

In [2]:
set_total = 1
# How many Set
ALL = True
if not ALL:
    set_num = 1
DEBUG = False

In [3]:
def load_data_num(path, istrain): # get the max num of leafnode and return
    df = pd.read_csv(path, sep='\t')
    num, index = node_num(df['Leafnode'])
    if istrain:
        max_label = df['Label'].max()
        return max(num), max_label
    else:
        return max(num)
    
def get_df(path):
    df = pd.read_csv(path, sep='\t')
    return df

def node_num(data): # generate a list of number of nodes that each page have
    count = False
    num_list = []
    for index in range(len(data)):
        if data[index] == 0 and count != False:
            num_list.append(data[index-1] + 1)
        else:
            count = True
    num_list.append(data[len(data) - 1] + 1)
    #print(num_list)
    count = 0
    index_list = []
    for i in num_list:
        if count == 0:
            index_list.append(i - 1)
            count += 1
        else:
            index_list.append(index_list[count - 1] + i)
            count += 1
    #print(index_list)
    return num_list, index_list

def label_padding(data, num): # padding the data with zero when that page is less than max_num leafnode
    #print(num)
    output = []
    count = 0
    for page_num in num:
        tmp = []
        page = 0
        #print('Page num: %s, Max: %s' %(page_num, max_num))
        if page_num == max_num:
            for i in range(page_num):
                tmp.append(data[count])
                count += 1
            #print('Num: %s, Count: %s' %(num[page], count))
            page += 1
        else:
            for i in range(page_num):
                tmp.append(data[count])
                count += 1
            for i in range(max_num - page_num):
                tmp.append(0) # Pad label with 0
            #print('Num: %s, Count: %s' %(num[page], count))
            page += 1
        output.append(tmp)
    return output

def node_data(data, num): # padding the data with zero when that page is less than max_num leafnode
    #print(num)
    output = []
    count = 0
    for page_num in num:
        tmp = []
        page = 0
        #print('Page num: %s, Max: %s' %(page_num, max_num))
        if page_num == max_num:
            for i in range(page_num):
                tmp.append(data[count])
                count += 1
            #print('Num: %s, Count: %s' %(num[page], count))
            page += 1
        else:
            for i in range(page_num):
                tmp.append(data[count])
                count += 1
            for i in range(max_num - page_num):
                tmp.append(9999)
            #print('Num: %s, Count: %s' %(num[page], count))
            page += 1
        output.append(tmp)
    return output

def load_data_csv(df): # load the csv file and convert it to np array
    num, index = node_num(df['Leafnode'])
    feature_1 = np.array(node_data(df['Leafnode'], num))
    df.drop(['Leafnode'], axis=1)
    feature_2 = np.array(node_data(df['PTypeSet'], num))
    #print(feature_2.shape)
    df.drop(['PTypeSet'], axis=1)
    feature_3 = np.array(node_data(df['TypeSet'], num))
    #print(feature_3.shape)
    df.drop(['TypeSet'], axis=1)
    feature_4 = np.array(node_data(df['Contentid'], num))
    #print(feature_4.shape)
    df.drop(['Contentid'], axis=1)
    feature_5 = np.array(node_data(df['Pathid'], num))
    #print(feature_5.shape)
    df.drop(['Pathid'], axis=1)
    feature_6 = np.array(node_data(df['Simseqid'], num))
    print(feature_6.shape)
    df.drop(['Simseqid'], axis=1)
    
    feature_1 = np.expand_dims(feature_1, -1)
    feature_2 = np.expand_dims(feature_2, -1)
    feature_3 = np.expand_dims(feature_3, -1)
    feature_4 = np.expand_dims(feature_4, -1)
    feature_5 = np.expand_dims(feature_5, -1)
    feature_6 = np.expand_dims(feature_6, -1)
    
    feature = np.concatenate([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6], -1)
    feature = np.reshape(feature, [feature_1.shape[0]*max_num, 6])
    label_array = np.array(label_padding(df['Label'], num))
    m_label = df['Label'].max()
    df.drop(['Label'], axis=1)
    #print(label_array)
    label = []
    for pages in tqdm(range(len(label_array))): # Loop each page
        page = []
        for node in range(len(label_array[pages])): # Loop each node
            node_label = []
            for label_t in range(max_label + 1): # Loop each label and a additional empty label ex.1~142 0 is empty
                if label_t == label_array[pages][node]:
                    node_label.append(1.0)
                else:
                    node_label.append(0.0)
            page.append(node_label)
        label.append(page)
    label = np.array(label)
    #feature = feature.astype(np.float32)
    #feature = np.reshape(feature, [len(label_array)*max_num, 6])
    label_array = label_array.flatten()
    #print(feature)
    #print(feature.shape)
    '''
        feature.shape = (None, Max_num, 6)
        path_emb.shape = (None, Max_num, 50)
        content_emb.shape = (None, Max_num, 50)
        label.shape =(None, Max_num, Max_label)
    '''
    return feature, label_array, m_label

def word2features(sent, i):
    leaf = sent[i][0]
    ptyp = sent[i][1]
    typ = sent[i][2]
    content = sent[i][3]
    pathid = sent[i][4]
    sim = sent[i][5]
    features = {
        'ptyp': ptyp,
        'typ': typ,
        'content': content,
        'pathid': pathid,
        'sim': sim,
    }
    if leaf != 0:
        ptyp1 = sent[i-1][1]
        typ1 = sent[i-1][2]
        content1 = sent[i-1][3]
        pathid1 = sent[i-1][4]
        sim1 = sent[i-1][5]
        features.update({
            '-1:ptyp': ptyp1,
            '-1:typ': typ1,
            '-1:content': content1,
            '-1:pathid': pathid1,
            '-1:sim': sim1,
        })
    else:
        features['BOL'] = True
    if i < len(sent)-1:
        next_leaf = sent[i+1][0]
        if next_leaf != 0:
            ptyp1 = sent[i + 1][1]
            typ1 = sent[i + 1][2]
            content1 = sent[i + 1][3]
            pathid1 = sent[i + 1][4]
            sim1 = sent[i + 1][5]
            features.update({
                '+1:ptyp': ptyp1,
                '+1:typ': typ1,
                '+1:content': content1,
                '+1:pathid': pathid1,
                '+1:sim': sim1,
            })
        else:
            features['EOL'] = True
    else:
        features['EOL'] = True
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(sent[i]) for i in range(len(sent))]

# Check max_num in train

In [4]:
max_num_train, max_label_train = load_data_num("./data/train_raw.csv", True)
max_num_test = load_data_num("./data/ytest_raw.csv", False)
max_num = max(max_num_train, max_num_test)
print(max_num_train)
print(max_num_test)

939
939


# Load Set index File

In [5]:
col_set_dict={}
if set_total > 0:
    Set_dict = {}
    with open("./data/Set_idx.txt", "r") as set_file:
        Set_dict = eval(set_file.readline())
    col_set_dict = dict(map(reversed, Set_dict.items()))
    print(Set_dict)

{1: 703}


# Load Train File

In [6]:
max_label = max_label_train
df = get_df("./data/train_raw.csv")
feature_train, label_train, out_train = load_data_csv(df)
page_num = int(len(feature_train)/max_num)
#input_shape = feature_train.shape[1]

  0%|          | 0/30 [00:00<?, ?it/s]

(30, 939)


100%|██████████| 30/30 [00:11<00:00,  2.63it/s]


In [7]:
print(page_num)

30


In [8]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=5000,
    all_possible_transitions=True
)
feature_train = feature_train.tolist()
label_train = label_train.tolist()
X_train = [sent2features(feature_train)]
#X_test = [sent2features(test_sent)]
y_train = [sent2labels(label_train)]
#y_test = [sent2labels(ga_sent)]

In [9]:
#print(y_train)

In [10]:
start = time.time()
crf.fit(X_train, y_train)
t = time.time()-start

# Load Test file

In [11]:
df = get_df("./data/ytest_raw.csv")
feature_test, label_test, out_test = load_data_csv(df)

feature_test = feature_test.tolist()
label_test = label_test.tolist()
X_test = [sent2features(feature_test)]
page_test = int(len(feature_test)/max_num)
print(page_test)
#X_test = [sent2features(test_sent)]

  0%|          | 0/30 [00:00<?, ?it/s]

(30, 939)


100%|██████████| 30/30 [00:10<00:00,  2.81it/s]


30


In [12]:
labels = list(crf.classes_)
ts_start = time.time()
y_pred = crf.predict(X_test)
ts = time.time()-ts_start
y_pred = np.array(y_pred)
result = np.reshape(y_pred, [page_test, max_num])
result = result.astype(np.int64)

# Read Column Type

In [13]:
col_type = []
with open("./data/TableA.txt", "r") as file:
    line = file.readline()
    slot = line.rstrip("\n").split("\t")
    while(slot[0]!="ColType"):
        line = file.readline()
        slot = line.rstrip("\n").split("\t")
    col_type = slot[1:]
print(col_type)

['MD', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT', 'MT

# File prediction output

In [14]:
Set_data = []
if DEBUG:
    with open("./crfsuite/data/predictions.csv", "w") as file: # Create prediction file
        for col in col_type: # loop to write the Col type
            file.write(col + "\t")
            print(col + "\t", end='')
        file.write("\n")
        for page in tqdm(range(len(result))): # Loop each page
            sets = []
            for label in range(max_label + 1): # Loop whole label
                print("Label: " + str(label))
                if label == 0:
                    continue
                empty = True
                isset = False
                data = []
                for node in range(len(result[page])):
                    if result[page][node] == label:
                        if empty == False and not isset:
                            print(" ", end='')
                            file.write(" ")
                        empty = False
                        if label in col_set_dict.keys() and set_total > 0: # That col is a Set
                            isset = True
                            data.append(node)
                            print("Append:" + str(node))
                        else:
                            print(str(node), end='')
                            file.write(str(node))
                if label in col_set_dict.keys() and set_total > 0: # That col is a Set
                    print(str(col_set_dict[label])+"-"+str(page), end='')
                    file.write(str(col_set_dict[label])+"-"+str(page))
                    sets.append(data)
                print("\t", end='')
                file.write("\t")
            print("")
            file.write("\n")
            print(data)
            Set_data.append(sets)
else:
    with open("./crfsuite/data/predictions.csv", "w") as file: # Create prediction file
        for col in col_type: # loop to write the Col type
            file.write(col + "\t")
        file.write("\n")
        for page in tqdm(range(len(result))): # Loop each page
            sets = []
            for label in range(max_label + 1): # Loop whole label
                if label == 0:
                    continue
                empty = True
                isset = False
                data = []
                for node in range(len(result[page])):
                    if result[page][node] == label:
                        if empty == False and not isset:
                            file.write(" ")
                        empty = False
                        if label in col_set_dict.keys() and set_total > 0: # That col is a Set
                            isset = True
                            data.append(node)
                        else:
                            file.write(str(node))
                if label in col_set_dict.keys() and set_total > 0: # That col is a Set
                    file.write(str(col_set_dict[label])+"-"+str(page))
                    sets.append(data)
                file.write("\t")
            file.write("\n")
            Set_data.append(sets)
print("Done!")

100%|██████████| 30/30 [00:12<00:00,  2.33it/s]

Done!


# Create Set data output for test

In [15]:
if set_total > 0:
    with open("./crfsuite/set/Set_data.txt", "w") as set_train_file:
        tmp = str(Set_data)
        set_train_file.write(tmp)
        if DEBUG:
            print(tmp)

# Create Set Train File

In [16]:
set_data_count = []
if set_total > 0:
    for set_t in range(set_total):
        with open("./data/Set-"+ str(set_t+1) +".txt", "r") as set_file:
            set_tmp = []
            output_name = "./crfsuite/set/Set-"+ str(set_t+1) +"_train_raw.csv"
            if DEBUG:
                print("Generating:" + output_name + "\n")
            output = open(output_name, "w")
            output.write("Leafnode\tPTypeSet\tTypeSet\tContentid\tPathid\tSimseqid\tLabel\n")
            line = set_file.readline()
            slot = line.rstrip("\n").split("\t")
            while(slot[0]!="ColType"): 
                line = set_file.readline()
                slot = line.rstrip("\n").split("\t")
            with open("./set/Set-"+ str(set_t+1) +"_coltype.txt", "w") as col_file:
                col_file.write(str(slot[1:]))
            line = set_file.readline() # First line of data
            page_num = 0
            count = 0
            while(line != ""):
                slot = line.rstrip("\n").split("\t")
                data_info = slot[0].split("-")
                if(page_num != int(data_info[1])):
                    set_tmp.append(count)
                    count = 0
                set_num = int(data_info[0])
                page_num = int(data_info[1])
                if DEBUG:
                    print(str(data_info[0])+"-"+str(data_info[1])+"-"+str(data_info[2]))
                idx = 1
                sub_list = slot[1:]
                while("" in sub_list):
                    sub_list.remove("")
                while(" " in sub_list):
                    sub_list.remove(" ")
                for element in sub_list:
                    count += 1
                    if DEBUG:
                        print(element)
                    element = int(element)
                    #print(content_train[page_num][element])
                    output.write(str(feature_train[page_num*max_num+element][0])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][1])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][2])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][3])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][4])+"\t")
                    output.write(str(feature_train[page_num*max_num+element][5])+"\t")
                    output.write(str(idx) + "\n")
                    if DEBUG:
                        print(feature_train[page_num*max_num+element][0])
                    idx += 1
                line = set_file.readline()
            set_tmp.append(count)
            output.close()
        set_data_count.append(set_tmp)

In [17]:
if set_total > 0:
    with open("./crfsuite/set/set_train_count.txt", "w") as file:
        file.write(str(set_data_count))
        print(set_data_count)

[[0, 10, 4, 5, 6, 4, 4, 4, 10, 1, 6, 4, 7, 13, 18, 1, 14, 4, 5, 1, 2]]


# Create Set Test file

In [18]:
set_data_count = []
if set_total > 0:
    for set_t in range(set_total):
        set_tmp = []
        with open("./crfsuite/set/Set-"+ str(set_t+1) +"_ytest_raw.csv", "w") as set_file:
            set_file.write("Leafnode\tPTypeSet\tTypeSet\tContentid\tPathid\tSimseqid\tLabel\n")
            for pages in tqdm(range(len(Set_data))):
                count = 0
                for node in Set_data[pages][set_t]:
                    count += 1
                    set_file.write(str(feature_train[pages*max_num+node][0])+"\t")
                    set_file.write(str(feature_train[pages*max_num+node][1])+"\t")
                    set_file.write(str(feature_train[pages*max_num+node][2])+"\t")
                    set_file.write(str(feature_train[pages*max_num+node][3])+"\t")
                    set_file.write(str(feature_train[pages*max_num+node][4])+"\t")
                    set_file.write(str(feature_train[pages*max_num+node][5])+"\t")
                    set_file.write(str(0) + "\n")
                set_tmp.append(count)
        set_data_count.append(set_tmp)

100%|██████████| 30/30 [00:00<00:00, 102466.71it/s]


In [19]:
if set_total > 0:
    with open("./crfsuite/set/set_test_count.txt", "w") as file:
        file.write(str(set_data_count))
        if DEBUG:
            print(set_data_count)

In [20]:
'''timef = open("./crfsuite/data/time_crf.txt","w")
print("\ntrain time:"+str(t))
timef.write("train:"+str(t)+"\n")
print("test time:"+str(ts))
print("per page:"+ str(float(ts)/len(result))+"\n")
timef.write("test:"+str(ts)+"\n")
timef.write("per page:"+ str(float(ts)/len(result))+"\n")
timef.close()'''
page_c = len(result)

# Set function define

In [21]:
def get_df(path):
    df = pd.read_csv(path, sep='\t')
    return df

def max_num_set(set_data_count, set_total):
    max_set = []
    for i in range(set_total):
        max_set.append(0)
    for sets in range(len(set_data_count)):
        max_set[sets] = max(set_data_count[sets])
    return max_set

def feature_padding(df, set_count, set_num):
    feature = []
    count = 0
    for pages in set_count[set_num-1]:
        set_len = pages
        for i in range(set_len):
            feature.append(df[count])
            count += 1
        if set_len != max_set[set_num-1]:
            for i in range(max_set[set_num-1]-set_len):
                feature.append(9999)
    return feature

def emb_padding(df, set_count, set_num, pad_len):
    emb = []
    tmp = []
    for i in range(pad_len):
        tmp.append(0)
    count = 0
    for pages in set_count[set_num-1]:
        set_len = pages
        for i in range(set_len):
            emb.append(eval(df[count]))
            count += 1
        if set_len != max_set[set_num-1]:
            for i in range(max_set[set_num-1]-set_len):
                emb.append(tmp)
    print(count)
    return emb

def one_of_n(ans, total):
    tmp = []
    for i in range(int(total)):
        if ans == i:
            tmp.append(1.0)
        else:
            tmp.append(0.0)
    return tmp

def label_padding(df, set_count, set_num):
    label = []
    #tmp = one_of_n(0, max_label+1)
    count = 0
    for pages in set_count[set_num-1]:
        set_len = pages
        for i in range(set_len):
            label.append(df[count])
            count += 1
        if set_len != max_set[set_num-1]:
            for i in range(max_set[set_num-1]-set_len):
                label.append(0)
    return label

def to_train_array(df, set_count, set_num):
    feature_1 = np.array(feature_padding(df['Leafnode'], set_count, set_num))
    feature_2 = np.array(feature_padding(df['PTypeSet'], set_count, set_num))
    feature_3 = np.array(feature_padding(df['TypeSet'], set_count, set_num))
    feature_4 = np.array(feature_padding(df['Contentid'], set_count, set_num))
    feature_5 = np.array(feature_padding(df['Pathid'], set_count, set_num))
    feature_6 = np.array(feature_padding(df['Simseqid'], set_count, set_num))
    
    feature_1 = np.expand_dims(feature_1, -1)
    feature_2 = np.expand_dims(feature_2, -1)
    feature_3 = np.expand_dims(feature_3, -1)
    feature_4 = np.expand_dims(feature_4, -1)
    feature_5 = np.expand_dims(feature_5, -1)
    feature_6 = np.expand_dims(feature_6, -1)
    
    feature = np.concatenate([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6], -1)
    feature = np.reshape(feature, [len(set_count[set_num-1])*max_set[set_num-1], 6])
    
    label = np.array(label_padding(df['Label'], set_count, set_num))
    label = np.reshape(label, [len(set_count[set_num-1])*max_set[set_num-1]])
    return feature, label

def word2features(sent, i):
    leaf = sent[i][0]
    ptyp = sent[i][1]
    typ = sent[i][2]
    content = sent[i][3]
    pathid = sent[i][4]
    sim = sent[i][5]
    features = {
        'ptyp': ptyp,
        'typ': typ,
        'content': content,
        'pathid': pathid,
        'sim': sim,
    }
    if leaf != 0:
        ptyp1 = sent[i-1][1]
        typ1 = sent[i-1][2]
        content1 = sent[i-1][3]
        pathid1 = sent[i-1][4]
        sim1 = sent[i-1][5]
        features.update({
            '-1:ptyp': ptyp1,
            '-1:typ': typ1,
            '-1:content': content1,
            '-1:pathid': pathid1,
            '-1:sim': sim1,
        })
    else:
        features['BOL'] = True
    if i < len(sent)-1:
        next_leaf = sent[i+1][0]
        if next_leaf != 0:
            ptyp1 = sent[i + 1][1]
            typ1 = sent[i + 1][2]
            content1 = sent[i + 1][3]
            pathid1 = sent[i + 1][4]
            sim1 = sent[i + 1][5]
            features.update({
                '+1:ptyp': ptyp1,
                '+1:typ': typ1,
                '+1:content': content1,
                '+1:pathid': pathid1,
                '+1:sim': sim1,
            })
        else:
            features['EOL'] = True
    else:
        features['EOL'] = True
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(sent[i]) for i in range(len(sent))]

# Read Set data

In [22]:
if set_total > 0:
    Set_data = []
    set_train_count = []
    set_test_count = []
    with open("./crfsuite/set/Set_data.txt", "r") as set_file:
        Set_data = eval(set_file.readline())
    with open("./crfsuite/set/set_train_count.txt", "r") as set_file:
        set_train_count = eval(set_file.readline())
    with open("./crfsuite/set/set_test_count.txt", "r") as set_file:
        set_test_count = eval(set_file.readline())
    max_num_train = max_num_set(set_train_count, set_total)
    max_num_test = max_num_set(set_test_count, set_total)
    max_set = []
    for i in range(len(max_num_train)):
        max_set.append(max(max_num_train[i], max_num_test[i]))
    if DEBUG:
        print(max_num_train)
        print(max_num_test)
        print(max_set)

# Run ALL

In [23]:
if set_total > 0:
    for num in range(set_total):
        set_num = num + 1
        # Load Train file & Test file
        df = get_df("./crfsuite/set/Set-"+str(set_num)+"_train_raw.csv")
        max_num = max_set[set_num-1]
        max_label = max(df['Label'])
        feature_train, label_train = to_train_array(df, set_train_count, set_num)
        page_num = int(len(feature_train)/max_num)
        crf = sklearn_crfsuite.CRF(
            algorithm='lbfgs',
            c1=0.1,
            c2=0.1,
            max_iterations=50000,
            all_possible_transitions=True
        )
        feature_train = feature_train.tolist()
        label_train = label_train.tolist()
        X_train = [sent2features(feature_train)]
        y_train = [sent2labels(label_train)]
        
        
        # Train
        start = time.time()
        crf.fit(X_train, y_train)
        t += time.time()-start
        
        # Save model
        
        
        #Load model
        
        
        # Load Test file
        df = get_df("./crfsuite/set/Set-"+str(set_num)+"_ytest_raw.csv")
        feature_test, label_test = to_train_array(df, set_test_count, set_num)
        feature_test = feature_test.tolist()
        label_test = label_test.tolist()
        X_test = [sent2features(feature_test)]
        page_test = int(len(feature_test)/max_num)
            
        # Prediction
        labels = list(crf.classes_)
        ts_start = time.time()
        y_pred = crf.predict(X_test)
        ts += time.time()-ts_start
        y_pred = np.array(y_pred)
        result = np.reshape(y_pred, [page_test, max_num])
        result = result.astype(np.int64)
        
        # Read Col
        col_type = []
        with open("./set/Set-"+str(set_num)+"_coltype.txt", "r") as file:
            tmp = file.readline()
            slot = eval(tmp)
            col_type = slot
            
        # Output
        Set = []
        if DEBUG:
            with open("./crfsuite/set/set-"+str(set_num)+".csv", "w") as file: # Create prediction file
                for col in col_type: # loop to write the Col type
                    file.write(col + "\t")
                    print(col + "\t", end='')
                print("")
                file.write("\n")
                current_pos = 1
                for page in tqdm(range(len(result))): # Loop each page
                    p_tmp = []
                    for cols in range(max_label+1):
                        c_tmp = []
                        for node in range(len(result[page])):
                            r = result[page][node]
                            if r == cols:
                                c_tmp.append(node)
                        p_tmp.append(c_tmp)
                    Set.append(p_tmp)
                Set_tmp = Set.copy()
                for page in range(len(Set_tmp)):
                    empty = False
                    col = []
                    for i in range(len(Set_tmp[page])):
                        col.append(False)
                    col[0] = True
                    while(not empty):
                        for cols in range(len(Set_tmp[page])):
                            if len(Set_tmp[page][cols]) == 0:
                                col[cols] = True
                                if cols != 0:
                                    print("\t", end="")
                                    file.write("\t")
                            else:
                                n = str(int(feature_test[page*max_num+Set_tmp[page][cols][0]][0]))
                                if cols != 0:
                                    print(n+"\t", end="")
                                    file.write(n+"\t")
                                del Set_tmp[page][cols][0]
                                if len(Set_tmp[page][cols]) == 0:
                                    col[cols] = True
                            empty = True
                            for i in col:
                                if i == False:
                                    empty = False
                                    break
                        print("\n", end="")
                        file.write("\n")
        else:
            with open("./crfsuite/set/set-"+str(set_num)+".csv", "w") as file: # Create prediction file
                for col in col_type: # loop to write the Col type
                    file.write(col + "\t")
                file.write("\n")
                current_pos = 1
                for page in tqdm(range(len(result))): # Loop each page
                    p_tmp = []
                    for cols in range(max_label+1):
                        c_tmp = []
                        for node in range(len(result[page])):
                            r = result[page][node]
                            if r == cols:
                                c_tmp.append(node)
                        p_tmp.append(c_tmp)
                    Set.append(p_tmp)
                Set_tmp = Set.copy()
                for page in range(len(Set_tmp)):
                    empty = False
                    col = []
                    for i in range(len(Set_tmp[page])):
                        col.append(False)
                    col[0] = True
                    while(not empty):
                        for cols in range(len(Set_tmp[page])):
                            if len(Set_tmp[page][cols]) == 0:
                                col[cols] = True
                                if cols != 0:
                                    file.write("\t")
                            else:
                                n = str(int(feature_test[page*max_num+Set_tmp[page][cols][0]][0]))
                                if cols != 0:
                                    file.write(n+"\t")
                                del Set_tmp[page][cols][0]
                                if len(Set_tmp[page][cols]) == 0:
                                    col[cols] = True
                            empty = True
                            for i in col:
                                if i == False:
                                    empty = False
                                    break
                        file.write("\n")

100%|██████████| 30/30 [00:00<00:00, 17394.13it/s]


In [24]:
timef = open("./crfsuite/data/time_crf.txt","w")
print("\ntrain time:"+str(t))
timef.write("train:"+str(t)+"\n")
print("test time:"+str(ts))
print("per page:"+ str(float(ts)/page_c)+"\n")
timef.write("test:"+str(ts)+"\n")
timef.write("per page:"+ str(float(ts)/page_c)+"\n")
timef.close()


train time:7650.392967224121
test time:39.63529944419861
per page:1.3211766481399536

